In [ ]:
! pip install datasets transformers[sentencepiece] sacrebleu

In [ ]:
import datasets
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("mkb", "en-te")
metric = load_metric("sacrebleu")

In [3]:
import transformers

print(transformers.__version__)

4.40.0


In [4]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-dra"

In [ ]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("pib", "en-te")
metric = load_metric("sacrebleu")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 44888
    })
})

In [ ]:
raw_datasets["train"]["translation"][0:2]

[{'en': "President's Secretariat SCHEDULE OF PRESIDENTS VISIT TO HYDERABAD The President of India, Shri Pranab Mukherjee will arrive in Hyderabad tomorrow at Begumept Airport by 12.05 PM.",
  'te': 'రాష్ట్రప\u200cతి స\u200cచివాల\u200cయం హైదరాబాద్ లో రాష్ట్రపతి పర్యటన షెడ్యూల్ భారత రాష్ట్రపతి శ్రీ ప్రణబ్ ముఖర్జీ బుధవారం మధ్యాహ్నం హైదరాబాద్ బేగంపేట విమానాశ్రయానికి చేరుకుంటారు.'},
 {'en': 'The President will address the inaugural function of Centenary Celebrations of Osmania University.',
  'te': 'అక్కడి నుండి నేరుగా ఉస్మానియా విశ్వవిద్యాలయం శత జయంతి ఉత్సవాల ప్రారంభ కార్యక్రమాలలో పాల్గొని ప్రసంగిస్తారు.'}]

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'NTPC is also efficiently managing the coal supplies for continuous supply of electricity.', 'te': 'నిరంతరాయ విద్యుత్ ఉత్పత్తి, సరఫరా కోసం బొగ్గు సరఫరాను కూడా NTPC సమర్థవంతంగా నిర్వహిస్తోంది.'}"
1,"{'en': 'The kit can be used for RNA extraction for RT-LAMP, RT-qPCR, RT-PCR and other isothermal and PCR based protocols for the detection of SARS-COV-2.', 'te': 'ఈ కిట్‌ను సార్స్‌-సిఒవి2 ను గుర్తించేందుకు ఆర్‌టి-ల్యాంప్‌, ఆర‌టి-క్యుపిసిఆర్‌, ఆర్‌టి-పిసిఆర్ ఇత‌ర ఐసోథ‌ర్మ‌ల్‌, పిసిఆర్ ఆధారిత ప్రోటోకాల్స్ కోసం ఆర్ ఎన్ ఎను వెలికితీయ‌డానికి ఉప‌యోగించ‌వ‌చ్చు.'}"
2,"{'en': 'Trains are being run by Railways only after concurrence is given both by the State which is sending the passengers and the State which is receiving them.These 800 trains were terminated in various states like Andhra Pradesh, Bihar, Chattisgarh, Himachal Pradesh, Jammu Kashmir, Jharkhand, Karnataka, Madhya Pradesh, Maharashtra, Manipur, Mizoram, Odisha, Rajasthan, Tamil Nadu, Telangana, Tripura, Uttar Pradesh, Uttarakhand, West Bengal.', 'te': 'కాగా, వలసకార్మికులను పంపించే, స్వీకరించే రాష్ట్రాల అంగీకారం మేరకే రైల్వేశాఖ ఈ ప్రత్యేక రైళ్లను నడుపుతోంది. తదనుగుణంగా ఇప్పటిదాకా నడిచిన 800 రైళ్లు ఆంధ్రప్రదేశ్, బీహార్, ఛత్తీస్‌గఢ్‌, హిమాచల్ ప్రదేశ్, జమ్ముకశ్మీర్‌, జార్ఖండ్, కర్ణాటక, మధ్యప్రదేశ్, మహారాష్ట్ర, మణిపూర్‌, మిజోరం, ఒడిశా, రాజస్థాన్, తమిళనాడు, తెలంగాణ, త్రిపుర, ఉత్తర ప్రదేశ్, ఉత్తరాఖండ్‌, పశ్చిమ బెంగాల్ రాష్ట్రాల్లోని గమ్యస్థానాలకు చేరాయి.'}"
3,"{'en': 'Major impact: Environmental degradation falls on the socially and economically disadvantaged, more heavily than the better of sections of the society.', 'te': 'ప్ర‌ధాన ప్ర‌భావం: ప‌ర్యావ‌ర‌ణం స్థాయి పడిపోతూ ఉన్న‌ప్పుడు స‌మాజం లోని ఉత్త‌మ వ‌ర్గాల క‌న్నా సామాజికం గా, ఆర్థికం గా వంచన కు గురి అయిన వ‌ర్గాల వారి పై కాస్తంత అధికమైనటువంటి ప్ర‌భావం పడుతుంది.'}"
4,"{'en': 'But the major reason is not being able to adapt to the upcoming changes in the future, especially in the field of technology.', 'te': 'ఇలా చేయగలిగినప్పుడు మాత్రమే మానవాళి ఇంధన అవసరాలను మనం అభిలషణీయ రీతిలో తీర్చగలం.'}"


In [ ]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: The system stream (a sequence of segments).
    references: A list of one or more reference streams (each a sequence of segments).
    smooth_method: The smoothing method to use. (Default: 'exp').
    smooth_value: The smoothing value. Only valid for 'floor' and 'add-k'. (Defaults: floor: 0.1, add-k: 1).
    tokenize: Tokenization method to use for BLEU. If not provided, defaults to 'zh' for Chinese, 'ja-mecab' for
        Japanese and '13a' (mteval) otherwise.
    lowercase: Lowercase the data. If True, enables case-insensitivity. (Default: False).
    force: Insist that your tokenized input is actually detokenized.

Returns:
    'score': BLEU score,
    'counts'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/799k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
from keras_preprocessing.text import tokenizer_from_json
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)

In [ ]:
tokenizer(">>tel<< Hello, this one sentence!")

{'input_ids': [11, 11171, 3, 62, 75, 11439, 49, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "te"
prefix = ""

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[27442, 53, 16, 18208, 50335, 88, 905, 553, 2279, 12266, 35326, 493, 3066, 46560, 397, 12137, 16238, 397, 61712, 3886, 2784, 4421, 1125, 493, 59625, 23390, 50, 27442, 5, 5251, 3, 33101, 3250, 1914, 8881, 9131, 15309, 26138, 434, 28063, 396, 38, 14402, 9, 58627, 3905, 28798, 8533, 80, 1282, 12883, 7171, 1034, 88, 52349, 59, 1633, 14546, 1914, 1213, 2, 0], [50, 27442, 38, 2698, 4, 9, 177, 23869, 42313, 7102, 5, 595, 5180, 1814, 6221, 595, 31887, 1059, 31637, 16, 5, 54610, 4616, 8226, 2, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[32003, 616, 22, 1432, 2421, 22, 3058, 198, 1623, 22, 4581, 31260, 985, 840, 3155, 2586, 2127, 32003, 616, 1432, 17, 18477, 883, 256, 12904, 769, 6890, 561, 17, 13647, 557, 32003, 616, 1432, 

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/45 [00:00<?, ?ba/s]

In [ ]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(40000))
eval_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/pib/en-te/1.3.0/385b8bfb3a894b6ec22185b138f24cb2911fddc8a2700ac99690ba4baa8cc43e/cache-be73589ee059baeb.arrow


In [ ]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
from transformers import MarianMTModel
model = MarianMTModel.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/294M [00:00<?, ?B/s]

In [ ]:
from transformers import  DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

Downloading:   0%|          | 0.00/294M [00:00<?, ?B/s]

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive

drive.mount("/content/drive")
torch.save(model, '/content/drive/MyDrive/Colab Notebooks/my_marianMT_model')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/my_marianMT_model')

In [ ]:
src_text = [
    ">>tel<< he is an intelligent boy",
    ">>tel<< what she is doing?",
    ">>tel<< he can sing a song",
]

In [ ]:
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

In [ ]:
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

['అతను తెలివిగల బాలుడు', 'ఆమె ఏమి చేస్తున్నారు?', 'అతను ఒక పాట పాడవచ్చు']

In [ ]:
tel_sen = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
!apt-get install libsox-fmt-all libsox-dev sox > /dev/null
! python -m pip install torchaudio > /dev/null
! python -m pip install git+https://github.com/facebookresearch/WavAugment.git > /dev/null

  Running command git clone -q https://github.com/facebookresearch/WavAugment.git /tmp/pip-req-build-7l0j668t


In [ ]:
!pip install ffmpeg-python > /dev/null

In [ ]:
!pip install pydub

In [ ]:
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [ ]:
# code taken from https://ricardodeazambuja.com/deep_learning/2019/03/09/audio_and_video_google_colab/
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import ffmpeg
import tempfile
import pathlib
from pydub import AudioSegment


AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  b = b64decode(data.split(',')[1])
  with open('audio.wav','wb') as f:
    f.write(b)
  wav_file = AudioSegment.from_file("audio.wav")
  wav_file = wav_file.set_frame_rate(16000)
  wav_file.export(out_f = "wav_file.wav", format = "wav")
  return wav_file

In [ ]:
import torchaudio
x = get_audio()

In [ ]:
from txtai.pipeline import Transcription

# Create transcription model
transcribe = Transcription("facebook/wav2vec2-large-960h")

In [ ]:
from IPython.display import Audio, display

files = "wav_file.wav"

text = transcribe(files)
display(Audio(files))
print(text)
print()


He is an indiligent boy



In [ ]:
translated = model.generate(**tokenizer(text, return_tensors="pt", padding=True))

In [ ]:
tel = tokenizer.decode(translated[0], skip_special_tokens=True)

In [ ]:
tel

'అతను తెలివిగల బాలుడు'

In [ ]:
! pip install gTTS

In [ ]:
from gtts import gTTS
from IPython.display import Audio
text_1 = tel
tel_language = 'te'

obj = gTTS(text=text_1, lang=tel_language, slow=False)
obj.save('tel_1.mp3')
sound_file = 'tel_1.mp3'
Audio(sound_file, autoplay=True)
